# "The Eye Of The Typer" Dataset Rerun

Visualize using `rerun`.

In [13]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [14]:
from pathlib import Path
from datetime import datetime, timedelta
from pprint import pprint

import polars as pl
import cv2 as cv

from eott_dataset import *

In [15]:
# %env set EOTT_DATASET_PATH
pdf = read_participant_characteristics()
participants = [Participant.create(**row) for row in pdf.iter_rows(named=True)]
pdf

pid,log_id,date,setting,display_width,display_height,screen_width,screen_height,distance_from_screen,screen_recording,wall_clock,gender,age,race,skin_color,eye_color,facial_hair,vision,touch_typer,handedness,weather,pointing_device,notes,time_of_day,duration,start_time
u8,u64,date,cat,u16,u16,f64,f64,f64,datetime[ms],datetime[μs],cat,u8,cat,cat,cat,cat,cat,bool,cat,cat,cat,str,time,duration[ms],datetime[ms]
1,1491423217564,2017-04-05,"""Laptop""",1440,900,33.17,20.73,60.0,2017-04-05 20:19:12.200,2017-04-05 20:36:00,"""Male""",25,"""Asian""","""1""","""Dark Brown to …","""None""","""Normal""",true,"""Right""","""Cloudy""","""Trackpad""","""Did not see th…",16:00:00,16h 40m,2017-04-05 20:13:37.564
2,1491487691210,2017-04-06,"""Laptop""",1440,900,33.17,20.73,55.0,2017-04-06 14:18:38.800,2017-04-06 14:41:00,"""Male""",22,"""Asian""","""1""","""Dark Brown to …","""None""","""Normal""",false,"""Right""","""Indoors""","""Trackpad""",null,10:00:00,22h 43m,2017-04-06 14:08:11.210
6,1491573819193,2017-04-07,"""PC""",1920,1200,51.7,32.31,77.0,null,null,"""Female""",25,"""White""","""1""","""Gray to Blue o…","""None""","""Normal""",true,"""Right""","""Sunny""","""Mouse""","""Trapped in one…",10:00:00,17h 5m,2017-04-07 14:03:39.193
7,1491577862552,2017-04-07,"""Laptop""",1440,900,33.17,20.73,null,2017-04-07 15:42:35.600,2017-04-07 16:03:00,"""Female""",27,"""Asian""","""4""","""Dark Brown to …","""None""","""Glasses""",false,"""Right""","""Sunny""","""Trackpad""","""No video for t…",11:00:00,20h 38m,2017-04-07 15:11:02.552
8,1491591937547,2017-04-07,"""PC""",1920,1200,51.7,32.31,83.0,null,null,"""Male""",23,"""White""","""1""","""Dark Brown to …","""Beard""","""Glasses""",true,"""Right""","""Cloudy""","""Mouse""",null,15:00:00,1d 9m,2017-04-07 19:05:37.547
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
60,1493219365171,2017-04-26,"""Laptop""",1440,900,33.17,20.73,58.0,2017-04-26 14:53:39.700,2017-04-26 15:13:00,"""Male""",27,"""Asian""","""2""","""Dark Brown to …","""Little""","""Glasses""",false,"""Right""","""Indoors""","""Mouse""",null,11:00:00,19h 5m,2017-04-26 15:09:25.171
61,1493222549175,2017-04-26,"""Laptop""",1440,900,33.17,20.73,54.0,2017-04-26 16:06:05.800,2017-04-26 16:22:00,"""Female""",29,"""Other""","""3""","""Dark Brown to …","""None""","""Glasses""",true,"""Right""","""Cloudy""","""Trackpad""","""Trapped in one…",12:00:00,15h 20m,2017-04-26 16:02:29.175
62,1493230037782,2017-04-26,"""Laptop""",1440,900,33.17,20.73,56.0,2017-04-26 18:13:06.100,2017-04-26 18:37:00,"""Male""",24,"""Black""","""5""","""Dark Brown to …","""Beard""","""Normal""",true,"""Right""","""Cloudy""","""Trackpad""",null,14:00:00,22h 51m,2017-04-26 18:07:17.782


In [16]:
import rerun as rr
from eott_dataset.rerun import *

p = participants[0]
study_type = Study.DOT_TEST

recording_id = f"{p.participant_id}/{study_type}"
timeline_df = get_timeline(p).filter(pl.col("study") == study_type)

rr.init("EOTT", recording_id=recording_id, spawn=True)
rr.log(
    "participant",
    rr.TextDocument(
        "\n".join(f"[{key}]\n{value}\n" for key, value in p.to_dict().items())
    ),
    timeless=True,
)

screen_cap: cv.VideoCapture | None = None
webcam_cap: cv.VideoCapture | None = None
webcam_path: Path | None = None

screen_size: tuple[int, int] = (p.display_width, p.display_height)
screen_size = tuple(map(lambda v: int(v * screen_scale), screen_size))
screen_factor = 2 if p.setting is Setting.LAPTOP else 1

webcam_scale = 1.0
screen_scale = 0.5

if p.screen_recording_path.exists():
    screen_cap = cv.VideoCapture(str(p.screen_recording_path))

rr.log(
    f"screen",
    rr.Boxes2D(sizes=[screen_size], centers=[tuple(map(lambda v: .5 * v, screen_size))]),
    timeless=True,
)

rr.log(
    "participant/pupil/left/tobii",
    rr.SeriesLine(name="left pupil diameter (tobii)", color=(255, 255, 0), width=1),
    timeless=True,
)
rr.log(
    "participant/pupil/right/tobii",
    rr.SeriesLine(name="right pupil diameter (tobii)", color=(255, 0, 255), width=1),
    timeless=True,
)

frame_index: int
webcam_index: int | None
study_name: str | None
offset_time: timedelta
source_name: Source
for log_index, (
    frame_index,
    webcam_index,
    study_name,
    source_name,
    offset_time,
) in enumerate(timeline_df.iter_rows()):
    study_name = Study(study_name) if study_name is not None else None

    rr.set_time_sequence("log_index", log_index)
    rr.set_time_seconds("capture_time", offset_time.total_seconds())

    timeline_name = f"webcam-{webcam_index}" if source_name == "webcam" else source_name
    rr.set_time_sequence(f"{timeline_name}_index", frame_index)

    match source_name:
        case "tobii":
            entry: TobiiEntry = p.tobii_gaze_predictions.row(frame_index, named=True)
            rerun_log_tobii(entry, screen=screen_size)

        case "screen" if screen_cap is not None:
            assert p.screen_offset is not None
            screen_cap = rerun_log_screen(screen_cap, position=frame_index, size=screen_size)

        case "webcam" if webcam_index is not None and study_name is not None:
            paths = p.get_webcam_video_paths(study=study_name, index=webcam_index)

            if len(paths) > 0 and (webcam_path != paths[0] or webcam_cap is None):
                webcam_path = paths[0]
                webcam_cap = cv.VideoCapture(str(webcam_path.absolute()))

            webcam_cap = rerun_log_webcam(webcam_cap, scale=webcam_scale)

        case "log":
            entry: LogEntry = p.user_interaction_logs.row(frame_index, named=True)
            rerun_log_user(entry, scale=screen_scale)

# end of logging
if screen_cap is not None:
    screen_cap.release()

if webcam_cap is not None:
    webcam_cap.release()

Found existing process on port 9876. Trying to connect.
